# Interneto sistemų, paslaugų kūrimas

Flask - "mažas" karkasas (angl. microframework) skirtas web sistemų kūrimui. Flask yra pagrįstas Werkzeug (WSGI biblioteka (angl. Web Server Gateway Interface)) ir Jinja 2 (šablonų variklis (angl. template engine)) bibliotekomis. Mažu šis karkasas vadinamas, todėl, jog šis karkasas yra ganėtinai "plikas", neturi kitiems karkasas būdingų funcionalumų, pvz. validatoriai, duomenų bazės palaikymas, migracijos ir t.t. Jūs gaunat pagrindą, o reikiamus funkcionalumus, struktūrą ir pan. kuriatės patys (diegiat reikiamus Flask papildymus).

In [ ]:
# aplikacijos pavyzdys
from flask import Flask # importuojame Flask modulį
app = Flask(__name__) # sukuriamas Flask objektas

@app.route("/") # sukuriama URL taisyklė, kurią pasiekus iškviečiama dekoruojama funkcija
def index(): # funkcija, kuri bus vykdoma pasiekus URL, dažnai vadinama view'u
    return("Hello World") # atsakas (angl. response) duomenys

if __name__ == "__main__":
    app.run() # app.run(host, port, debug, options)

Norint užregistruoti funkciją, kuri bus iškviečiama pasiekus tam tikrą URL, funkcija dekoruojama dekoratoriumi @app.route("/"). Be šio dekoratoriaus tą patį rezultatą galima pasiekti ir panaudojus funkciją app.add_url_rule("/", "index", index)

In [ ]:
# aplikacijos pavyzdys
from flask import Flask # importuojame Flask modulį
app = Flask(__name__) # sukuriamas Flask objektas

def index():
    return("Hello World") # atsakas (angl. response) duomenys

app.add_url_rule("/", "index", index) # sukuriama URL taisyklė, kurią pasiekus iškviečiama nurodyta funkcija

if __name__ == "__main__":
    app.run() # app.run(host, port, debug, options)

In [ ]:
# galima sukurti kiek tik reikia URL taisyklių, taip pat vienai funkcijai nurodyti kelias URL taisykles
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
    return("""
    <!DOCTYPE html>
      <html>
      <head><title>Hello</title></head>
      <body><h1>Hello, from HTML</h1></body>
      </html>
    """)

@app.route("/career/")
def career():
    return("Career Page")

@app.route("/feedback/")
def feedback():
    return("Feedback Page")

@app.route('/first/first/')
@app.route('/second/first/')
def numbers():
    return("Multiple URLS")

if __name__ == "__main__":
    app.run()

### Dinaminiai URL, parametrai

Norint sukurti dinaminius URL (funkcijai perduoti parametrus) aprašant URL naudojamas sekantis formatas: "/user/<id>/". Toks užrašymas nurodo kad funkcijai bus perduodamas parametras id, kurio tipas yra bet koks. Toks URL užrašymas įvykdys funkciją kai bus pasiekiamas adresas /users/1/, /users/100/, /users/pirmas/, /users/antras/. Kaip matom id laukas galės įgyti tiek int tiek string tipo reikšmes. Dėl to galima naudoti tokį URL formatą <konvertatorius:kintamojo pavadinimas>, kuris nurodo kokio tipo kintamojo tikimės, galimos reikšmės: string, int, float, path, uuid.

In [ ]:
# dinaminis URL
from flask import Flask

app = Flask(__name__)

@app.route("/user/<id>/")
def user_profile(id):
    return("Profile page of user #{}".format(id))

if __name__ == "__main__":
    app.run()

In [ ]:
# dinaminis URL su parametro tipo tikrinimu
from flask import Flask

app = Flask(__name__)

@app.route("/user/<int:id>/")
def user_profile(id):
    return("Profile page of user #{}".format(id))

if __name__ == "__main__":
    app.run()

### Debug būsena

Pagal nutylėjimą debug būsena yra išjungta. Jei įvyksta klaida programiniame kode, vartotojui grąžinamas 500 atsakas, be papildomų paaiškinimų. Įsijungus debug būseną į ekraną grąžinama pilna informacija apie klaidą. Taip pat jei įjungta ši būsena, po kiekvieno failo pakoregavimo ir išsaugojimo serveris yra perleidžiamas iš naujo.

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def index():
    print(i)
    return("Index")

if __name__ == "__main__":
    app.run(debug=True) # True/False

### URL ir funkcijų sąrašas

app objekta turi metodą app.url_map, kuris grąžina sąrašą URL->funkcija susiejimų.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
    return("Home Page")

@app.route("/career/")
def career():
    return("Career Page")

@app.route("/feedback/")
def feedback():
    return("Feedback Page")

@app.route('/first/first/')
@app.route('/second/first/')
def numbers():
    return("Multiple URLS")

print(app.url_map)

### Kontekstas (angl. contexts) Flask karkase

Kaip pastebėjote, visose view funkcijose nėra paduodamas pačios užklausos (angl. request) parametras. Šis parametras yra pasiekiamas visose funkcijose. Gali susidaryti įspūdis, jog jis yra globalus, tačiau taip nėra, nes daugiagijėse (angl. multithread) programose kiltų problemų (vienu metu paleistos kelios applikacijos, pvz. naudojant DispatcherMiddleware), kokia užklausa priklauso kokiai gijai (angl. thread), todėl Flask naudoja kintamuosius, kurie yra lokalūs kiekvienai gijai (angl. thread locals), turi savo kontekstą.

Flask turi kelis skirtingus kontekstus: aplikacijos ir užklausos. Aplikacijos kontekstas skirtas saugoti bendrus duomenis, tokius kaip duomenų bazės prisijungimus, kitus nustatymus, o užklausos kontekste saugomi duomenys skirti kiekvienai užklausai.

http://flask.pocoo.org/docs/0.12/appcontext/
http://flask.pocoo.org/docs/0.12/reqcontext/

Flask karkase aplikacijos ir užklausos kontekstai yra aktyvuojami/sukuriami kai gaunama nauja užklausa, ir deaktyvuojami, kai užklausa yra apdorota.

In [ ]:
# užklausos konteksto pavyzdys
from flask import Flask, request

app = Flask(__name__)

@app.route("/")
def requestdata():
    return("Hello! Your IP is {} and you are using {}: ".format(request.remote_addr, request.user_agent))

if __name__ == "__main__":
    app.run()

### Užklausos atsakymas (angl. response)

Flask karkase yra trys atsakymo tipai:
1. Paprastas tekstas arba naudojant šablonų variklį.
2. Response objektas.
3. Sąrašas iš (atsakyams, būsena, antraštės) arba (atsakymas, antraštės/būsena) (angl. (response, status, headers) or (response, headers/status)).

Kai grąžinamas tekstas Flask automatiškai tą tekstą paverčia į Response objektą (naudojant funkciją make_response()) su HTTP būsena 200 ir antrašte content-type = text/html. Norint pakeisti grąžinamus parametrus, galima išsikviesti make_response() funkciją ir atlikti jo grąžinto objekto korekcijas.

In [ ]:
# make_response funkcijos pavyzdys
from flask import Flask, make_response

@app.route("/books/<genre>")
def books(genre):
    res = make_response("All Books in {} category".format(genre))
    res.headers["Content-Type"] = "text/plain"
    res.headers["Server"] = "VU"
    return(res)

In [ ]:
# grąžinamas atsakymas su būsena 400
@app.route("/")
def http_404_handler():
    return make_response("<h2>404 Error</h2>", 400)

In [ ]:
# sausainėlių (angl. cookies) nustatymas
@app.route("/set-cookie")
def set_cookie():
    res = make_response("Cookie setter")    
    res.set_cookie("favorite-color", "skyblue")
    res.set_cookie("favorite-font", "sans-serif")
    # res.set_cookie("favorite-color", "skyblue", 60*60*24*15) # cookies su galiojimo laiku
    # res.set_cookie("favorite-font", "sans-serif", 60*60*24*15) # cookies su galiojimo laiku
    return res

In [ ]:
# atsakymas kaip sąrašas
@app.route("/")
def http_500_handler():
    return ("<h2>500 Error</h2>", 500)
    # return("## Heading", 200, {'Content-Type': 'text/markdown'}) # grąžinamas atsakymas su antraštėmis

### Užklausos peradresavimas

In [ ]:
@app.route("/transfer")
def transfer():
    return "", 302, {"location": "http://localhost:5000/login"}

In [ ]:
from flask import Flask, redirect

@app.route("/transfer")
def transfer():
    return redirect("http://localhost:5000/login")
    # return redirect("http://localhost:5000/login", code=301) # peradresavimas su būsena

### Spąstų vietos (angl. hook points)

Dažnai web aplikacijose reikia atlikti tam tikrus veiksmus prieš kiekvieną užklausą, pvz. kliento IP adreso užloginimas, arba vartotojo autentikavimas prieš parodant jiems turinį ir pan. Flask turi kelis dekoratorius, kurie gali pagelbėti atlikti šiuos veiksmus:

* before_first_request - vykdys funkciją prieš pirmą užklausą.
* before_request - vykdys funkciją prieš kiekvieną užklausą.
* after_request - vykdys funkciją po užklausos vykdymo. Nėra vykdoma įvykus nesuvaldytai klaidai (angl. unhandled exception). Funkcija privalo priimti atsakymo objektą (angl. response objext), o grąžinti tą patį arba naują atsakymo objektą.
* teardown_request - panašu į after_request dekoratorių, tik šį funkciją bus vykdoma visada nepaisant ar buvo klaida ar ne.

In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.before_first_request
def before_first_request():
    print("before_first_request() called")

@app.before_request
def before_request():
    print("before_request() called")

@app.after_request
def after_request(response):
    print("after_request() called")
    return(response)

@app.route("/")
def index():
    print("index() called")
    return("<p>Testings Request Hooks</p>")

@app.teardown_request
def teardown_request(response):
    print("teardown_request() called")
    return(response)

if __name__ == "__main__":
    app.run(debug=True)

### Užklausų atšaukimas

Užklausų atšaukimui naudojama funkcija abort() su grąžinamu atsakymo kodu. Tokiu atveju grąžinamas tas pats atsakymas skirtingų klaidų atveju. Norint sukurti atskirus klaidų atsakymus naudojamas errorhandler dekoratorius.

In [ ]:
from flask import Flask, abort

@app.route("/")
def index():
    abort(404)
    # tolimesnis kodas nebus vykdomas

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
@app.errorhandler(404)
def http_404_handler(error):
    return("<p>HTTP 404 Error Encountered</p>", 404)

@app.errorhandler(500)
def http_500_handler(error):
    return("<p>HTTP 500 Error Encountered</p>", 500)